In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("histopathologic-cancer-detection/train_labels.csv")
df.label.value_counts()

label
0    130908
1     89117
Name: count, dtype: int64

In [3]:
130908/89117

1.468945319074924

In [4]:

def split_dataframe(df:pd.DataFrame, num_of_splits:int) -> list[pd.DataFrame]:

    # Shuffle the rows
    df_shuffled = df.sample(frac=1, random_state=10) 

    rows_per_split = len(df_shuffled) // num_of_splits

    dfs = [df_shuffled.iloc[i * rows_per_split:(i + 1) * rows_per_split] for i in range(num_of_splits)]

    # Print to check line losses
    print(f'Original dataset has {len(df)} rows')
    print(f'Splitted has in total {sum([len(part) for part in dfs])} rows')

    return dfs

dfs = split_dataframe(df = df, num_of_splits = 5)

Original dataset has 220025 rows
Splitted has in total 220025 rows


In [5]:
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
#from torchvision import datasets, transforms
#from torch.autograd import Variable
#from torch.optim import Optimizer
#from torch.utils import data
import pretrainedmodels
import pandas as pd
#import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import os
#import cv2
from skimage.io import imread
#from torch.utils.data.sampler import WeightedRandomSampler, BatchSampler
#from tqdm import tqdm
#from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
#import pretrainedmodels.utils as utils
#from sklearn.metrics import roc_auc_score
from tools import *

def write_log(logfile, train_loss, test_loss, test_score, lr):
    with open(logfile, "a+") as log:
        log.write("{}\t{}\t{}\t{}\n".format(train_loss, test_loss, test_score, lr))


In [6]:

# PER IMPOSTARE IL DEVICE
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print('MPS is available')
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available')
else:
    device = torch.device('cpu')
    print('No acceleration available')


MPS is available


In [7]:

# FOLDERS PATH
DIR = 'histopathologic-cancer-detection/'
train_im_dir = DIR+'/train'
test_im_dir = DIR+'/test'

# IMPORTING DATA
train_data = pd.read_csv(os.path.join(DIR,'train_labels.csv')) #labels for train data


train_data.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [8]:
# MODELS
# model_dir = os.path.join(DIR, 'resnet34')
# model_name = 'resnet34'
model_dir = os.path.join(DIR, 'this_model')
model_name = 'this_model'

In [9]:

n_groups = 15 # number of folds # CROSS-VALIDATION PER VALIDAZIONE
b_size = 96 # batch size


In [10]:
# CROSS VALIDATION K-FOLD 
skf = StratifiedKFold(n_splits=n_groups)

# CONTENITORI PER RACCOGLIERE I DATI DELLA CROSS-VALIDATION
folds_id_train = []
folds_label_train = []
folds_id_val = []
folds_label_val = []

# RIEMPIO I CONTENITORI PER LA CROSS-VALIDATION
for train_index, test_index in skf.split(train_data['id'].values, train_data['label'].values):
    # Train
    folds_id_train.append(train_data['id'].values[train_index])
    folds_label_train.append(train_data['label'].values[train_index])
    # Validation
    folds_id_val.append(train_data['id'].values[test_index])    
    folds_label_val.append(train_data['label'].values[test_index])



In [11]:
print(len(folds_id_train))
print(len(folds_id_train[0]))

print(len(folds_id_val))
print(len(folds_id_val[0]))

15
205356
15
14669


In [12]:

# INIZIO DEL TRAINING
samples_per_epoch = 50000 #define number of samples per epoch, since dataset is big
# CICLO SUI GRUPPI DELLA C.V.
for valid_idx in range(n_groups):

    logfile =  model_dir+'/{}.fold{}.logfile.txt'.format(model_name, valid_idx)
    best_w_path = model_dir+'/{}.fold{}.best.pt'.format(model_name, valid_idx)
    es_w_path =  model_dir+'/{}.fold{}.es.pt'.format(model_name, valid_idx)
    
    
    print('Training fold {}'.format(valid_idx))
    
    #with open(logfile, "w") as log:
    #    pass    
    
    traing_aug = aug_train() # FUNZIONE IN UTILS
    validation_aug = aug_val() # FUNZIONE IN UTILS
    
    curr_lr = 3e-4
    
    train_sampler = torch.utils.data.RandomSampler(DataGenerator(folds_id_val[valid_idx],       # GENERATES DATASET FOR LOADING
                                                                 folds_label_val[valid_idx], 
                                                                 validation_aug, train_im_dir),
                                                   replacement=True, 
                                                   num_samples=samples_per_epoch)
    
    train_loader = torch.utils.data.DataLoader(DataGenerator(folds_id_train[valid_idx], 
                                                             folds_label_train[valid_idx], 
                                                             traing_aug, train_im_dir),
                                               pin_memory=False,
                                               num_workers=4,
                                               batch_size=b_size, 
                                               sampler=train_sampler)
    
    val_loader = torch.utils.data.DataLoader(DataGenerator(folds_id_val[valid_idx], 
                                                       folds_label_val[valid_idx], 
                                                       validation_aug, train_im_dir),
                                             pin_memory=False,
                                             num_workers=1,
                                             batch_size=b_size)
    
    # DEFINIAMO LA FUNZIONE DI LOSS - 
    loss_f = nn.BCELoss() # BINARY CROSS ENTROPY

    best_score = 0
    best_loss = 1e5
    idx_stop = 0

    ##############################################
    # OK QUI BISOGNA LAVORARCI

    # LOAD RESNET34 - PRETRAINED  # questo andrebbe sostituito cazzo!
    base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                           pretrained='imagenet').to(device) 
    
    #model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  # Esempio di ResNet-18 preaddestrata

    
    # DEFINISCO IL MODEL! # OK QUI 
    model = Net(base_model, 512).to(device)

    ##############################################
    
    # "TRAINING WITH FROZEN LAYERS EXCEPT FOR CLASSIFICATION HEAD"
    # OK, ALCUNI LAYERS VENGONO BLOCCATI CON UN LEARNING RATE PARI A ZERO, PER CUI NON VENGONO RIADDESTRATI!
    optimizer = optim.SGD([{'params': model.layer0.parameters(), 'lr': 0},
                           {'params': model.layer1.parameters(), 'lr': 0},
                           {'params': model.layer2.parameters(), 'lr': 0},
                           {'params': model.layer3.parameters(), 'lr': 0},
                           {'params': model.layer4.parameters(), 'lr': 0},
                           {'params': model.classif.parameters()}], lr=0.05, momentum=0.9)
    

    
    # QUI INIZIA GIà A TRAINARE, QUESTO è IL PRIMO GIRO, CHISSà PERCHè LO FA ESTERNO
    # LAUNCH TRAIN, TEST AND WRITE LOG
    train_loss = train(model= model,
                           train_loader= train_loader, 
                           optimizer= optimizer, 
                           epoch= 0, 
                           log_interval= 100, 
                           loss_f= loss_f, 
                           samples_per_epoch= samples_per_epoch,
                           scheduler= None)
    
    test_loss, score = test(model= model, 
                                val_loader= val_loader, 
                                loss_f= loss_f)
    
    write_log(logfile, train_loss, test_loss, score, lr = "not available")
    
    '''
    start training the model with all layers
    Training scheme : train while validation loss decreases, save model at each improvement of test loss. 
    if loss does not decreases for 3 epochs, reload last best model, reduce lr by factor of 2. 
    If loss still doesn't decrease for 10 epochs, stop the model. 
    '''
    for epoch in range(50):
        print(f'we are in epoch {epoch}')
        optimizer = torch.optim.SGD(model.parameters(), lr=curr_lr, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=curr_lr, max_lr=3*curr_lr, mode = 'triangular')
        #scheduler = CyclicLR(optimizer, max_lr=3*curr_lr)

        # LAUNCH TRAIN, TEST AND WRITE LOG
        train_loss = train(model= model,
                           train_loader= train_loader, 
                           optimizer= optimizer, 
                           epoch= epoch, 
                           log_interval= 100, 
                           loss_f= loss_f, 
                           samples_per_epoch= samples_per_epoch,
                           scheduler= scheduler)
        
        test_loss, score = test(model= model, 
                                test_loader= val_loader, 
                                loss_f= loss_f)
        
        write_log(logfile, train_loss, test_loss, score)
        
        if test_loss<best_loss:
            print('Test loss improved from {} to {}, saving'.format(best_loss, test_loss))
            best_loss = test_loss
            torch.save(model.state_dict(), best_w_path)
            idx_stop = 0
        else:
            print('Loss {}, did not improve from {} for {} epochs'.format(test_loss, best_loss, idx_stop))
            idx_stop += 1
        if idx_stop>3:
            print('Reducing LR by two and reloading best model')
            model.load_state_dict(torch.load(best_w_path))
            curr_lr = curr_lr/2
        if idx_stop>10:
            print('Stopping the model')
            torch.save(model.state_dict(), es_w_path)


Training fold 0


/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gabrielegabrielli/Library/Caches/pypoetry/virtualenvs/neural-networks-_F4AaA2c-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Setting DEVICE:
	 MPS is available
Train Epoch: 0 [0/50000 (0.000%)]	Loss: 0.704052
Train Epoch: 0 [9600/50000 (19.200%)]	Loss: 0.704023
Train Epoch: 0 [19200/50000 (38.400%)]	Loss: 0.702323
Train Epoch: 0 [28800/50000 (57.600%)]	Loss: 0.704685
Train Epoch: 0 [38400/50000 (76.800%)]	Loss: 0.702748
Train Epoch: 0 [48000/50000 (96.000%)]	Loss: 0.703608
Mean train loss on epoch 0 : 0.7035729429125785


TypeError: test() got an unexpected keyword argument 'val_loader'